In [1]:
import os

In [2]:
%pwd

'/Users/srinidhikankanala/Desktop/ML E2E Project in VS/Latest ML Project E2E/E2E Project MLFlow/End-to-end-Machine-Learning-Project-with-MLflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/srinidhikankanala/Desktop/ML E2E Project in VS/Latest ML Project E2E/E2E Project MLFlow/End-to-end-Machine-Learning-Project-with-MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    # the above parameters are from config.yaml file data_ingestion part

in normal class we use self keyword
in dataclass we dont need self keyword and also we just assign values to variables and we dont do any functions so we use dataclass

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        # these are coming form src/constatnts/__init__.py file

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [9]:
# Component-Data Ingestion component
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    # DOWNLOADING THE ZIP FILE
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-18 15:32:13,533: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-18 15:32:13,535: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-18 15:32:13,536: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-18 15:32:13,536: INFO: common: created directory at: artifacts]
[2025-03-18 15:32:13,537: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-18 15:32:14,400: INFO: 1594274967: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B38C:3CD2AD:588D85:6CBA7F:67D9D84D
Accept-Ranges: bytes
Date: Tue, 18